In [1]:

import argparse
from pathlib import Path
import os
import glob
import cv2
import numpy as np
from tqdm import tqdm
import fnmatch
from face_detection import FaceDetection
from params import RANDOM_SEED, LocationConfig, CreateDataConfig

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}


/home/karol/miniconda3/envs/ur/lib/python3.8/site-packages/onnxruntime/capi/onnxruntime_inference_collection.py:55: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'CPUExecutionProvider'
  warnings.warn("Specified provider '{}' is not in available provider names."


In [26]:
def load_img(path: Path) -> np.ndarray:
    if not path.exists():
        raise Exception(f"Given file: '{path}' - does not exists!")
    return cv2.imread(str(path))

In [15]:
def squarify(img):
    old_image_height, old_image_width, channels = img.shape
    if min(old_image_height, old_image_width) < 10:
        return None
    max_size = max(old_image_height, old_image_width)
    if max_size < 38:
        return None
    color = (255,255,255)
    result = np.full((max_size,max_size, channels), color, dtype=np.uint8)

    # compute center offset
    x_center = (max_size - old_image_width) // 2
    y_center = (max_size - old_image_height) // 2
    
    x_error = (max_size - old_image_width) % 2
    y_error = (max_size - old_image_height) % 2

    # copy img image into center of result image
    result[y_center:y_center+old_image_height+x_error, 
           x_center:x_center+old_image_width+y_error] = img
    
    return result

In [39]:
list_of_face_shapes = [[],[]]

In [40]:
def detect_and_crop(face_detect_app: FaceDetection, input_path: Path, output_path: str):
    img = load_img(input_path)
    faces = face_detect_app.detect_face(img)
    if len(faces) == 0 or len(faces) > 1:
        return

    face_box = faces[0].astype(np.int32)
    try:
        new_img = img[face_box[1] : face_box[3], face_box[0] : face_box[2]]
    except IndexError:
        return
    list_of_face_shapes[0].append(new_img.shape[0])
    list_of_face_shapes[1].append(new_img.shape[1])
    # new_img = squarify(new_img)
    if new_img is None:
        return
    
    filename = str(input_path).split('/')[-1]
    # cv2.imwrite(os.path.join(output_path,filename), new_img)

In [41]:
face_detector = FaceDetection()

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}


/home/karol/miniconda3/envs/ur/lib/python3.8/site-packages/onnxruntime/capi/onnxruntime_inference_collection.py:55: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'CPUExecutionProvider'
  warnings.warn("Specified provider '{}' is not in available provider names."


In [42]:
total_f=0
for image_path in tqdm(Path(LocationConfig.raw_data).glob('*/*.jpg')):
    total_f += 1

30935it [00:00, 275942.81it/s]


In [43]:
for image_path in tqdm(Path(LocationConfig.raw_data).glob('*/*.jpg'), total=total_f):
    detect_and_crop(face_detector, image_path, LocationConfig.crop_data)

  0%|          | 11/30935 [00:03<2:40:05,  3.22it/s]


KeyboardInterrupt: 